# MMS SITL Ground Loop: Figure 4
This notebook reproduces Figure 4 of **Argall, et al. 2020**.

## Front matter

In [ ]:
import pathlib
import datetime as dt
from pymms.sdc import selections as sel
import matplotlib.pyplot as plt

## Input Parameters
The figure plots data from SROI 1 only between 19 Oct. 2019, which is when the GLS was fully implemented, and 25 Mar. 2020, a few days before the paper was submitted. To reproduce the plots in the supplemental material, set `sroi=None` and `sroi=3`.

In [ ]:
sroi = 1
outdir = '~/Desktop/mp-dl-unh/'
start_date = dt.datetime(2019, 10, 19)
end_date = dt.datetime(2020, 3, 25, 18, 44, 46)

# Filter selections to be within a particular SROI
do_sroi=False
if sroi in (1, 2, 3):
    do_sroi=True

# Location to save the figure
if outdir is not None:
    outdir = pathlib.Path(outdir).expanduser()

## Metric
A function to calculate and histogram the overlap between selections.

In [ ]:
def plot_metric(ref_data, test_data, fig, labels, location,
                nbins=10):
    '''
    Visualize the overlap between segments.

    Parameters
    ----------
    ref_data : list of `BurstSegment`s
        Reference burst segments
    test_data : list of `BurstSegment`s
        Test burst segments. Determine which test segments
        overlap with the reference segments and by how much
    labels : tuple of str
        Labels for the reference and test segments
    location : tuple
        Location of the figure (row, col, nrows, ncols)
    nbins : int
        Number of histogram bins to create

    Returns:
    --------
    ax : `matplotlib.pyplot.Axes`
        Axes in which data is displayed
    ref_test_data : list of `BurstSegment`s
        Reference data that falls within the [start, stop] times
        of the test data.
    '''

    # Determine by how much the test data overlaps with
    # the reference data.
    ref_test = []
    ref_test_data = []
    ref_test = [selection_overlap(segment, test_data)
                for segment in ref_data]

    # Overlap statistics
    #   - Number of segments selected
    #   - Percentage of segments selected
    #   - Percent overlap from each segment
    ref_test_selected = sum(selection['n_selections'] > 0
                            for selection in ref_test)
    ref_test_pct_selected = ref_test_selected / len(ref_test) * 100.0
    ref_test_pct_overlap = [selection['pct_overlap'] for selection in ref_test]

    # Calculate the plot index from the (row,col) subplot location
    plot_idx = lambda rowcol, ncols : (rowcol[0]-1)*ncols + rowcol[1]

    # Create a figure
    ax = fig.add_subplot(location[2], location[3],
                         plot_idx(location[0:2], location[3]))
    hh = ax.hist(ref_test_pct_overlap, bins=nbins, range=(0, 100))
    #ax.set_xlabel('% Overlap Between {0} and {1} Segments'.format(*labels))
    if location[0] == location[2]:
        ax.set_xlabel('% Overlap per Segment')
    if location[1] == 1:
        ax.set_ylabel('Occurrence')
    ax.text(0.5, 0.98, '{0:4.1f}% of {1:d}'
              .format(ref_test_pct_selected, len(ref_test)),
              verticalalignment='top', horizontalalignment='center',
              transform=ax.transAxes)
    ax.set_title('{0} Segments\nSelected by {1}'.format(*labels))

    return ax, ref_test

Determine how much overlap one burst segment of a given class has with another class of burst segments.

In [ ]:
def selection_overlap(ref, tests):
    '''
    Gather overlap statistics.
    
    Parameters
    ----------
    ref : `selections.BurstSegment`
        The reference burst segment.
    tests : list of `selections.BurstSegment`
        The burst segements against which the reference segment is compared.
    
    Returns
    -------
    out : dict
        Data regarding how much the reference segment overlaps with the
        test segments.
    '''
    out = {'dt': ref.tstop - ref.tstart,
           'dt_next': dt.timedelta(days=7000),
           'n_selections': 0,
           't_overlap': dt.timedelta(seconds=0.0),
           't_overselect': dt.timedelta(seconds=0.0),
           'pct_overlap': 0.0,
           'pct_overselect': 0.0
           }

    # Find which selections overlap with the given entry and by how much
    tdelta = dt.timedelta(days=7000)
    for test in tests:

        if ((test.tstart <= ref.tstop) and
            (test.tstop >= ref.tstart)
            ):
            out['n_selections'] += 1
            out['t_overlap'] += (min(test.tstop, ref.tstop)
                                 - max(test.tstart, ref.tstart)
                                 )

        # Time to nearest interval
        out['dt_next'] = min(out['dt_next'], abs(test.tstart - ref.tstart))

    # Overlap and over-selection statistics
    if out['n_selections'] > 0:
        out['t_overselect'] = out['dt'] - out['t_overlap']
        out['pct_overlap'] = out['t_overlap'] / out['dt'] * 100.0
        out['pct_overselect'] = out['t_overselect'] / out['dt'] * 100.0
    else:
        out['t_overselect'] = out['dt']
        out['pct_overselect'] = 100.0

    return out

## Burst Selections
Get the ABS, GLS, and SITL burst selections. Filter by SROI and determine which SITL selections were classified as magnetopause crossings.

In [ ]:
abs_data = sel.selections('abs', start_date, end_date,
                          latest=True, combine=True, metadata=do_sroi)

gls_data = sel.selections('mp-dl-unh', start_date, end_date,
                          latest=True, combine=True, metadata=do_sroi)

sitl_data = sel.selections('sitl+back', start_date, end_date,
                           latest=True, combine=True, metadata=do_sroi)

# Filter by SROI
if do_sroi:
    abs_data = [s for s in abs_data if s.sroi == sroi]
    sitl_data = [s for s in sitl_data if s.sroi == sroi]
    gls_data = [s for s in gls_data if s.sroi == sroi]

sitl_mp_data = sel.filter_segments(sitl_data, '(MP|Magnetopause)')

Save the selections for reference.

In [ ]:
if outdir is not None:
    sroi_str = ''
    if do_sroi:
        sroi_str = '_sroi{0:d}'.format(sroi)
    
    for stype, data in zip(('sitl', 'abs', 'gls'), (sitl_data, abs_data, gls_data)):
        csv_fname = (outdir
                     / '_'.join((stype + sroi_str,
                                 start_date.strftime('%Y%m%d%H%M%S'),
                                 end_date.strftime('%Y%m%d%H%M%S')
                                 )))
        csv_fname = csv_fname.with_suffix('.csv')
        sel.write_csv(csv_fname, data)

## Create the Figure
Here, we create the figure. `plot_metric` will determine the overlap between each class of selections, plot a histogram of the results, and annotate the figure with summary statistics.

In [ ]:
# Create a figure
nbins = 10
nrows = 4
ncols = 3
fig = plt.figure(figsize=(8.5, 10))
fig.subplots_adjust(hspace=0.55, wspace=0.3)

# GLS-SITL Comparison
ax, gls_sitl = plot_metric(gls_data, sitl_data, fig,
                           ('GLS', 'SITL'), (1, 1, nrows, ncols),
                           nbins=nbins)
ax, sitl_gls = plot_metric(sitl_data, gls_data, fig,
                           ('SITL', 'GLS'), (2, 1, nrows, ncols),
                           nbins=nbins)
ax, gls_sitl_mp = plot_metric(gls_data, sitl_mp_data, fig,
                              ('GLS', 'SITL MP'), (3, 1, nrows, ncols),
                              nbins=nbins)
ax, sitl_mp_gls = plot_metric(sitl_mp_data, gls_data, fig,
                              ('SITL MP', 'GLS'), (4, 1, nrows, ncols),
                              nbins=nbins)

# ABS-SITL Comparison
ax, abs_sitl = plot_metric(abs_data, sitl_data, fig,
                           ('ABS', 'SITL'), (1, 2, nrows, ncols),
                           nbins=nbins)
ax, sitl_abs = plot_metric(sitl_data, abs_data, fig,
                           ('SITL', 'ABS'), (2, 2, nrows, ncols),
                           nbins=nbins)
ax, abs_sitl_mp = plot_metric(abs_data, sitl_mp_data, fig,
                              ('ABS', 'SITL MP'), (3, 2, nrows, ncols),
                              nbins=nbins)
ax, sitl_mp_abs = plot_metric(sitl_mp_data, abs_data, fig,
                              ('SITL MP', 'ABS'), (4, 2, nrows, ncols),
                              nbins=nbins)

# GLS-ABS Comparison
abs_mp_data = [abs_data[idx]
               for idx, s in enumerate(abs_sitl_mp)
               if s['n_selections'] > 0]

ax, gls_abs = plot_metric(gls_data, abs_data, fig,
                          ('GLS', 'ABS'), (1, 3, nrows, ncols),
                          nbins=nbins)
ax, abs_gls = plot_metric(abs_data, gls_data, fig,
                          ('ABS', 'GLS'), (2, 3, nrows, ncols),
                          nbins=nbins)
ax, gls_abs_mp = plot_metric(gls_data, abs_mp_data, fig,
                             ('GLS', 'ABS MP'), (3, 3, nrows, ncols),
                             nbins=nbins)
ax, abs_mp_gls = plot_metric(abs_mp_data, gls_data, fig,
                             ('ABS MP', 'GLS'), (4, 3, nrows, ncols),
                             nbins=nbins)

# Save the figure
if outdir is not None:
    sroi_str = ''
    if do_sroi:
        sroi_str = '_sroi{0:d}'.format(sroi)
    filename = (outdir
                / '_'.join(('selections_metric' + sroi_str,
                            start_date.strftime('%Y%m%d%H%M%S'),
                            end_date.strftime('%Y%m%d%H%M%S')
                            )))
    filename = filename.with_suffix('.png')
    plt.savefig(filename)